In [ ]:
import copy
import random
from tqdm import tqdm

import alifedata_phyloinformatics_convert as apc
from deap import algorithms as deap_algorithms
from deap import base as deap_base
from deap import creator as deap_creator
from deap import tools as deap_tools
from hstrat import hstrat

import pylib


# Setup


In [ ]:
deap_creator.create("FitnessMax", deap_base.Fitness, weights=(1.0,))
deap_creator.create("Individual", list, fitness=deap_creator.FitnessMax)

toolbox = deap_base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register(
    "individual",
    deap_tools.initRepeat,
    deap_creator.Individual,
    toolbox.attr_bool,
    n=100,
)
toolbox.register("population", deap_tools.initRepeat, list, toolbox.individual)


def evalOneMax(individual):
    return (sum(individual),)


toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", deap_tools.cxTwoPoint)
toolbox.register("mutate", deap_tools.mutFlipBit, indpb=0.05)
toolbox.register("select", deap_tools.selTournament, tournsize=2)

# Decorate the variation operators
history = deap_tools.History()
toolbox.decorate("mate", history.decorator)

toolbox.decorate("mate", pylib.deap.hstrat_mate_decorator)


# Evolve


In [ ]:
# Create the population and populate the history
population = toolbox.population(n=16)

common_ancestor = hstrat.HereditaryStratigraphicColumn(
    stratum_retention_policy=hstrat.perfect_resolution_algo.Policy(),
    stratum_differentia_bit_width=64,
    initial_stratum_annotation=0,
)
for member in population:
    member.species_annotation = common_ancestor.CloneDescendant()
    member.gene_annotation = common_ancestor.CloneDescendant(0)
history.update(population)

# Do the evolution, the decorators will take care of updating the
# history
NGEN = 10
for gen in tqdm(range(NGEN)):
    offspring = deap_algorithms.varAnd(
        population, toolbox, cxpb=1.0, mutpb=1.0
    )
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
    population = [copy.deepcopy(ind) for ind in population]

NGEN = 10
slice_size = len(population) // 2
from_subpopulations = []
for i in range(0, len(population), slice_size):
    subpopulation = population[i : i + slice_size]
    for gen in tqdm(range(NGEN)):
        offspring = deap_algorithms.varAnd(
            subpopulation, toolbox, cxpb=1.0, mutpb=1.0
        )
        fits = toolbox.map(toolbox.evaluate, offspring)
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit
        subpopulation = toolbox.select(offspring, k=len(subpopulation))
        subpopulation = [copy.deepcopy(ind) for ind in subpopulation]

    from_subpopulations.extend(subpopulation)

for x in from_subpopulations:
    history.update([x])

population = from_subpopulations


# Reconstruction


In [ ]:
extant_nodes = [individual.history_index for individual in population]
reconstructed_df = hstrat.build_tree(
    [individual.species_annotation for individual in population],
    version_pin=hstrat.__version__,
    force_common_ancestry=True,
    taxon_labels=map(str, extant_nodes),
)
reconstructed_df["name"] = reconstructed_df["taxon_label"]
reconstructed_df


In [ ]:
reconstructed_tree = apc.alife_dataframe_to_biopython_tree(
    reconstructed_df, setup_branch_lengths=True
)
_ = pylib.tree.draw_biopython_tree(reconstructed_tree)


In [ ]:
pylib.dag.draw_pedigree(
    pylib.dag.prune_dag(pylib.deap.history_to_dag(history), extant_nodes),
    extant_nodes,
)


# Actual Pedigree


In [ ]:
distilled_tree = pylib.deap.history_to_tree_upgma(history, extant_nodes)


In [ ]:
_ = pylib.tree.draw_biopython_tree(distilled_tree)


In [ ]:
pylib.tree.polymorphic_quartet_distance(reconstructed_tree, distilled_tree)
